In [1]:
import os
import json
import requests
import pandas as pd

from serpapi import GoogleSearch

from dotenv import load_dotenv

from tqdm.notebook import tqdm
tqdm.pandas()

## SERPAPI Setup
Setting up the SERPAPI API Pull

In [2]:
# Pulls key from .env 
SERPAPI_KEY = os.getenv("serpapi_key")

In [3]:
# Fetches Google Trends data since 2004 for a given keyword and country code - saves it to a JSON file.

# Not finished - parameters need to be set up to pull time series data in "data_type" and country code specification needs to be added
# Currently isn't pulling correct information

def fetch_google_trends(country_code, keyword, SERPAPI_KEY):    
    
    # Keeps the file names uniform - changing artist_name from here on
    keyword_filename = keyword

    # Replace underscores with spaces in the artist's name
    keyword = keyword_filename.replace("_", " ")
    
    # Parameters for the API call
    params = {
        "engine": "google_trends",
        "q": keyword,
        "data_type": "GEO_MAP_0",
        "date": "all",  # Specify the time range
        "api_key": SERPAPI_KEY , # Replace with your SerpAPI key
        "region": "COUNTRY",
        "geo": country_code , 
    }
    
    # Fetch data using SerpAPI
    search = GoogleSearch(params)
    results = search.get_dict()
    
    # Extract 'interest_over_time' section
    interest_over_time = results.get("interest_over_time", {})
    
    # Define the output file path
    output_path = f"../data/gtrends/{keyword}_{country_code}_Trends.json"

    # Save the data as a JSON file
    with open(output_path, "w") as file:
        json.dump(interest_over_time, file, indent=4)
    
    return print(f"Google Trends data successfully saved to {output_path}")

In [4]:
# Example
# fetch_google_trends("GB", "Chess", SERPAPI_KEY)

### FIDE Webscraper Setup
Here, we are setting up the FIDE webscraper to get a list of the current top 10 chess players

### Obtaining Chess.com Data

Finally, we are defining a function to output player data based on the list of top 10 players, obtained through the FIDE webscraper

In [5]:
# Testing cURL for the player endpoint
! curl -v https://api.chess.com/pub/player/hikaru

*   Trying 104.18.138.67:443...
* Connected to api.chess.com (104.18.138.67) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*  CAfile: /etc/ssl/cert.pem
*  CApath: none
* (304) (OUT), TLS handshake, Client hello (1):
* (304) (IN), TLS handshake, Server hello (2):
* (304) (IN), TLS handshake, Unknown (8):
* (304) (IN), TLS handshake, Certificate (11):
* (304) (IN), TLS handshake, CERT verify (15):
* (304) (IN), TLS handshake, Finished (20):
* (304) (OUT), TLS handshake, Finished (20):
* SSL connection using TLSv1.3 / AEAD-CHACHA20-POLY1305-SHA256
* ALPN, server accepted to use h2
* Server certificate:
*  subject: CN=chess.com
*  start date: Dec  3 07:03:24 2024 GMT
*  expire date: Mar  3 07:03:23 2025 GMT
*  subjectAltName: host "api.chess.com" matched cert's "*.chess.com"
*  issuer: C=US; O=Google Trust Services; CN=WE1
*  SSL certificate verify ok.
* Using HTTP2, server supports multiplexing
* Connection state changed (HTTP